In [15]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

In [16]:
def load_cifar_file(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [17]:
def features_reshape(feature_matrix):
    return np.reshape(feature_matrix,(10000,3,32,32)).transpose(0, 2, 3, 1)

In [18]:
def preprocess_data(folder_path):
    data_train=[]
    features_train=[]
    labels_train=[]
    for i in range(1,6):
        data_train.append(load_cifar_file(folder_path+'/cifar-10-batches-py/data_batch_'+str(i)))
        features_train.append(data_train[i-1][b'data'])
        labels_train.append(data_train[i-1][b'labels'])
    data_test=load_cifar_file('Downloads/cifar-10-batches-py/test_batch')
    features_test = data_test[b'data']
    lab_test = data_test[b'labels']
    
    features_rs=[]
    for i in range(5):
        features_rs.append(features_reshape(features_train[i]))
    labels=labels_train
    features_test=features_reshape(features_test)
    
    features_rs_train=[]
    features_rs_valid=[]
    labels_train=[]
    labels_valid=[]
    for idx,feat in enumerate(features_rs):
        features_rs_train.append(minmaxNormalize(feat[:2000]))
        features_rs_valid.append(minmaxNormalize(feat[2000:]))
    features_rs_test=minmaxNormalize(features_test)
    for idx in range(len(labels)):
        lab=labels[idx]
        labels_train.append(onehotLabels(lab[:2000]))
        labels_valid.append(onehotLabels(lab[2000:]))
    labels_test=onehotLabels(lab_test)
    
    with open('features_train.pkl', 'wb') as f:
        pickle.dump(features_rs_train, f)
    with open('features_valid.pkl', 'wb') as f:
        pickle.dump(features_rs_valid, f)
    with open('features_test.pkl', 'wb') as f:
        pickle.dump(features_rs_test, f)
    with open('labels_train.pkl', 'wb') as f:
        pickle.dump(labels_train, f)
    with open('labels_valid.pkl', 'wb') as f:
        pickle.dump(labels_valid, f)
    with open('labels_test.pkl', 'wb') as f:
        pickle.dump(labels_test, f)

In [19]:
#min-max normalization
def minmaxNormalize(feature_mat_list):
    for i in range(len(feature_mat_list)):
        feature_mat_list[i]=(feature_mat_list[i]-np.amin(feature_mat_list[i]))/(np.amax(feature_mat_list[i])-np.amin(feature_mat_list[i]))
    return feature_mat_list

In [20]:
def onehotLabels(lab):
    enc=OneHotEncoder(handle_unknown='ignore')
    enc.fit(np.asarray(lab).reshape(-1,1))
    lab_onehot=enc.transform(np.asarray(lab).reshape(-1,1))
    return lab_onehot